In [1]:
import dask.dataframe as dd
import dask.distributed

In [2]:
import pandas as pd

In [ ]:
client = dask.distributed.Client()

In [3]:
trips = dd.read_parquet('/bigdata/all_trips.parquet')

In [4]:
trips.columns

Index(['dropoff_datetime', 'dropoff_latitude', 'dropoff_location_id',
       'dropoff_longitude', 'ehail_fee', 'extra', 'fare_amount',
       'improvement_surcharge', 'mta_tax', 'passenger_count', 'payment_type',
       'pickup_datetime', 'pickup_latitude', 'pickup_location_id',
       'pickup_longitude', 'rate_code_id', 'store_and_fwd_flag', 'tip_amount',
       'tolls_amount', 'total_amount', 'trip_distance', 'trip_type',
       'vendor_id'],
      dtype='object')

In [5]:
import uuid, sqlalchemy

In [6]:
engine = sqlalchemy.create_engine(open('/home/shekhar/.sqlconninfo').read())
conn = engine.connect()

In [7]:
uu = uuid.uuid1().hex
tableID = 'uu_{}'.format(uu)
tableIDLoc = 'uuloc_{}'.format(uu)

In [8]:
uu

'a84ed8622f4711e79d2d902b3437c844'

In [9]:
# trips1 = trips.get_partition(1).compute()

In [10]:
# trips1499 = trips.get_partition(1499).compute()

In [11]:
# trips1 = trips1.append(trips1499)

In [48]:
trips1 = trips.get_partition(1).compute()

In [49]:
trips1 = trips1.reset_index(drop=True)

In [50]:
trips2 = trips1[['pickup_longitude', 'pickup_latitude', 'pickup_location_id']]
trips2.columns = ['lon', 'lat', 'locid']
trips2.to_sql(tableID, engine, index_label='trip_id')

In [51]:
trips2.head()

,lon,lat,locid
0,-73.9706,40.7586,NaN
1,-73.9994,40.7605,NaN
2,-73.9999,40.7320,NaN
3,-73.9793,40.7635,NaN
4,-74.0047,40.7204,NaN


In [38]:
trips1.shape

(1930590, 23)

In [39]:
conn.execute('''CREATE UNLOGGED TABLE {} AS
SELECT
  trip_id,
  ST_SetSRID(ST_MakePoint(lon, lat), 4326) as loc
FROM {}
WHERE locid IS NULL
;
CREATE INDEX on {} USING GIST(loc);
'''.format(tableIDLoc, tableID, tableIDLoc))

In [40]:
df1 = pd.read_sql('''SELECT t.trip_id, n.gid as census_tract_id
FROM {} AS t, nyct2010 AS n
WHERE ST_Within(t.loc, n.geom) ORDER BY t.trip_id;'''.format(tableIDLoc), engine)
df2 = pd.read_sql('''SELECT t.trip_id, n.gid as taxi_zone_id
FROM {} AS t, taxi_zones AS n
WHERE ST_Within(t.loc, n.geom) ORDER BY t.trip_id;'''.format(tableIDLoc), engine)

In [41]:
trips1.head()

,dropoff_datetime,dropoff_latitude,dropoff_location_id,dropoff_longitude,ehail_fee,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,...,pickup_location_id,pickup_longitude,rate_code_id,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_type,vendor_id
0,2016-12-26 15:04:45,NaN,144.0,NaN,NaN,0.0,13.5,0.3,0.5,1,...,186.0,NaN,1,N,1.00,0.0,15.30,3.10,yellow,1
1,2016-12-26 15:00:49,NaN,239.0,NaN,NaN,0.0,10.5,0.3,0.5,1,...,141.0,NaN,1,N,2.82,0.0,14.12,2.06,yellow,2
2,2016-12-26 15:03:00,NaN,262.0,NaN,NaN,0.0,15.0,0.3,0.5,2,...,79.0,NaN,1,N,0.00,0.0,15.80,4.26,yellow,2
3,2016-12-26 14:50:33,NaN,234.0,NaN,NaN,0.0,3.5,0.3,0.5,1,...,234.0,NaN,1,N,0.00,0.0,4.30,0.30,yellow,1
4,2016-12-26 14:58:17,NaN,163.0,NaN,NaN,0.0,8.0,0.3,0.5,1,...,170.0,NaN,1,N,1.76,0.0,10.56,1.29,yellow,2


In [42]:
trips2.head()

,lon,lat,locid
0,NaN,NaN,186.0
1,NaN,NaN,141.0
2,NaN,NaN,79.0
3,NaN,NaN,234.0
4,NaN,NaN,170.0


In [43]:
df1.set_index('trip_id').head()

,census_tract_id
trip_id,


In [44]:
df2.set_index('trip_id').head()

,taxi_zone_id
trip_id,


In [45]:
zz = (trips1.dropna(axis=1, how='all').merge(
        df1.set_index('trip_id'), left_index=True, right_index=True, how='left', sort=True)).merge(
        df2.set_index('trip_id'), left_index=True, right_index=True, how='left', sort=True)

In [46]:
# zz = (trips1.merge(df1, left_index=True, right_on='trip_id', how='left')).merge(
#     df2, left_index=True, right_on='trip_id', how='left')
zz.drop([ 'vendor_id', 'store_and_fwd_flag', 'payment_type'], axis=1).sort_index().rename(
    columns={'census_tract_id': 'pickup_ct_id', 'taxi_zone_id': 'pickup_tz_id'})

,dropoff_datetime,dropoff_location_id,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,pickup_datetime,pickup_location_id,rate_code_id,tip_amount,tolls_amount,total_amount,trip_distance,trip_type,pickup_ct_id,pickup_tz_id
0,2016-12-26 15:04:45,144.0,0.0,13.5,0.3,0.5,1,2016-12-26 14:48:26,186.0,1,1.00,0.0,15.30,3.10,yellow,NaN,NaN
1,2016-12-26 15:00:49,239.0,0.0,10.5,0.3,0.5,1,2016-12-26 14:48:26,141.0,1,2.82,0.0,14.12,2.06,yellow,NaN,NaN
2,2016-12-26 15:03:00,262.0,0.0,15.0,0.3,0.5,2,2016-12-26 14:48:26,79.0,1,0.00,0.0,15.80,4.26,yellow,NaN,NaN
3,2016-12-26 14:50:33,234.0,0.0,3.5,0.3,0.5,1,2016-12-26 14:48:27,234.0,1,0.00,0.0,4.30,0.30,yellow,NaN,NaN
4,2016-12-26 14:58:17,163.0,0.0,8.0,0.3,0.5,1,2016-12-26 14:48:27,170.0,1,1.76,0.0,10.56,1.29,yellow,NaN,NaN
5,2016-12-26 15:01:22,230.0,0.0,9.0,0.3,0.5,1,2016-12-26 14:48:27,164.0,1,1.96,0.0,11.76,0.88,yellow,NaN,NaN
6,2016-12-26 15:10:13,13.0,0.0,18.5,0.3,0.5,1,2016-12-26 14:48:27,230.0,1,0.00,0.0,19.30,4.64,yellow,NaN,NaN
7,2016-12-26 14:53:33,160.0,0.0,5.0,0.3,0.5,1,2016-12-26 14:48:28,82.0,1,0.00,0.0,5.80,0.70,yellow,NaN,NaN
8,2016-12-26 14:55:10,236.0,0.0,7.0,0.3,0.5,1,2016-12-26 14:48:28,237.0,1,1.17,0.0,8.97,1.20,yellow,NaN,NaN
9,2016-12-26 15:00:17,162.0,0.0,9.5,0.3,0.5,1,2016-12-26 14:48:28,237.0,1,2.00,0.0,12.30,1.40,yellow,NaN,NaN


In [47]:
engine.connect().execute('drop table {}; drop table {};'.format(tableID, tableIDLoc))

In [ ]:
def assign_taxi_zones(df, lon_var, lat_var, locid_var):
    """Joins DataFrame with Taxi Zones shapefile.

    This function takes longitude values provided by `lon_var`, and latitude
    values provided by `lat_var` in DataFrame `df`, and performs a spatial join
    with the NYC taxi_zones shapefile. 

    The shapefile is hard coded in, as this function makes a hard assumption of
    latitude and longitude coordinates. It also assumes latitude=0 and 
    longitude=0 is not a datapoint that can exist in your dataset. Which is 
    reasonable for a dataset of New York, but bad for a global dataset.

    Only rows where `df.lon_var`, `df.lat_var` are reasonably near New York,
    and `df.locid_var` is set to np.nan are updated. 

    Parameters
    ----------
    df : pandas.DataFrame or dask.DataFrame
        DataFrame containing latitudes, longitudes, and location_id columns.
    lon_var : string
        Name of column in `df` containing longitude values. Invalid values 
        should be np.nan.
    lat_var : string
        Name of column in `df` containing latitude values. Invalid values 
        should be np.nan
    locid_var : string
        Name of column in `df` containing taxi_zone location ids. Rows with
        valid, nonzero values are not overwritten. 
    """

    localdf = df[[lon_var, lat_var, locid_var]].copy()
    # localdf = localdf.reset_index()
    localdf[lon_var] = localdf[lon_var].fillna(value=0.)
    localdf[lat_var] = localdf[lat_var].fillna(value=0.)
    localdf['replace_locid'] = (localdf[locid_var].isnull()
                                & (localdf[lon_var] != 0.)
                                & (localdf[lat_var] != 0.))
    
    
    import sqlalchemy, uuid, os
    engine = sqlalchemy.create_engine(open(os.path.expanduser('~/.sqlconninfo')).read())
    
    tableID = 'uu{}'.format(uuid.uuid1().hex)
    localdf.to_sql(tableID, engine)
    
    conn = engine.connect()
    
    conn.execute()

#     if (np.any(localdf['replace_locid'])):
#         shape_df = geopandas.read_file('../shapefiles/taxi_zones_latlon.shp')
#         shape_df.drop(['OBJECTID', "Shape_Area", "Shape_Leng", "borough", "zone"],
#                       axis=1, inplace=True)

#         try:
#             local_gdf = geopandas.GeoDataFrame(
#                 localdf, crs={'init': 'epsg:4326'},
#                 geometry=[Point(xy) for xy in
#                           zip(localdf[lon_var], localdf[lat_var])])

#             local_gdf = geopandas.sjoin(
#                 local_gdf, shape_df, how='left', op='intersects')

#             # one point can intersect more than one zone -- for example if on
#             # the boundary between two zones. Deduplicate by taking first valid.
#             local_gdf = local_gdf[~local_gdf.index.duplicated(keep='first')]

#             local_gdf.LocationID.values[~local_gdf.replace_locid] = (
#                 (local_gdf[locid_var])[~local_gdf.replace_locid]).values

#             return local_gdf.LocationID.rename(locid_var).astype(np.float64)
#         except ValueError as ve:
#             print(ve)
#             print(ve.stacktrace())
#             return df[locid_var]
#     else:
#         return df[locid_var]